## Feature Engineering

ЦЕЛЬ

Всё та же: выяснить, какие регионы наиболее привлекательны для открытия офлайн-контор.

ЗАДАЧИ

Вспомним часть вопросов с прошедшего модуля:

1. Сколько раз человеку надо прийти, чтобы сделать ставку?
2. Каков средний выигрыш?
3. Каков баланс по каждому пользователю?
4. Какие города самые выгодные?
5. В каких городах самая высокая ставка?
6. Сколько в среднем времени занимает у человека с первого посещения сайта до первой попытки?
7. Мы постараемся ответить не только на них, но и на многие другие вопросы.

КОНКРЕТНЫЕ ШАГИ (ФОРМАЛИЗОВАННЫЕ ЗАДАЧИ)

* Вычислить, с какого раза пользователь делает ставку.
* Вычислить среднее время между первым посещением и первой ставкой.
* Определить средний выигрыш в процентах.
* Для каждого пользователя рассчитать баланс.
* Выявить, в каких городах делают самые большие ставки.

In [77]:
import pandas as pd

log = pd.read_csv('data/log.csv', header = None)
log.columns = ["user_id", "time", "bet", "win"]
users = pd.read_csv('data/users.csv', sep="\t", encoding="koi8_r")
users.columns = ["user_id", "email", "geo"]

In [78]:
def correct_user_id (new_id):
    if new_id == '#error':
        return ''
    else:
        return new_id[24:]

log = log.query('user_id != "#error"')
log['user_id'] = log.user_id.apply(correct_user_id)
log.time = log.time.str.strip('[')

### Пропущенные значения

В Pandas есть метод isna(), который возвращает таблицу такой же размерности, что и на вход, но значения в ней — True или False (True, если данное значение является пропуском, и False — в ином случае).

Важно! Метод isna() есть не только у DataFrame, но и у Series. Это значит, что применять его можно не только ко всей таблице, но и к каждому столбцу отдельно.

В numpy пропущенные значения могут быть записаны как специальный объект np.nan, что означает Not a Number.

Проверить наличие таких значений можно с помощью np.isnan().

In [37]:
log.head().isna() 

,user_id,time,bet,win
0,False,False,True,True
1,False,False,True,True
2,False,False,True,True
3,False,False,True,True
4,False,False,True,True


In [38]:
log['time'].isna().sum()

0

Удалять данные с пропусками можно с помощью метода dropna().

1. Параметр axis

Параметр axis в методе dropna() говорит методу, по какой оси удалять значения.

* Если нужно удалить строки, в которых встречается пропуск (NaN), нужно указывать axis=0.
* Если нужно удалить столбцы, в которых встречается пропуск (NaN), нужно указывать axis=1.

2. Параметр subset

Что он делает? Если передать в него список значений по одной оси (например, названия столбцов) и задать при этом в параметре axis другую ось (в нашем случае 0), то мы удалим те строки, для которых в данных столбцах находится пропуск.

То же самое работает и наоборот: нужно поменять axis на 1 и вместо названий столбцов передавать индексы строк.

Важно! Перед удалением строк обязательно сделайте бэкап.

In [39]:
log.head()

,user_id,time,bet,win
0,user_919,2019-01-01 14:06:51,NaN,NaN
1,user_973,2019-01-01 14:51:16,NaN,NaN
2,user_903,2019-01-01 16:31:16,NaN,NaN
3,user_954,2019-01-01 17:17:51,NaN,NaN
4,user_954,2019-01-01 21:31:18,NaN,NaN


In [123]:
log2 = log.copy()
log2.dropna(axis=1)

,user_id,time
0,user_919,[2019-01-01 14:06:51
1,user_973,[2019-01-01 14:51:16
2,user_903,[2019-01-01 16:31:16
3,user_954,[2019-01-01 17:17:51
4,user_954,[2019-01-01 21:31:18
...,...,...
991,user_965,[2019-04-20 12:55:41
992,user_967,[2019-04-20 14:59:36
993,user_973,[2019-04-20 17:09:56
994,user_977,[2019-04-20 18:10:07


### Дубли

Дубликаты (дубли) — это повторяющиеся строки в данных.

В Pandas есть метод для удаления дублей — drop_duplicates(). Он просто удаляет повторяющиеся строки.

У этого метода тоже есть параметр subset, но в этом случае ему нужно передавать список названий столбцов, а не их номера.

Одна из проблем при изучении библиотеки Pandas — её неконсистентность. Например, недавно мы передавали номера столбцов в параметр subset, а сейчас мы передаём названия столбцов. Это нужно просто запомнить.

In [4]:
log2 = log.copy()
log2.drop_duplicates(subset=['time','user_id'])

,user_id,time,bet,win
0,user_919,2019-01-01 14:06:51,NaN,NaN
1,user_973,2019-01-01 14:51:16,NaN,NaN
2,user_903,2019-01-01 16:31:16,NaN,NaN
3,user_954,2019-01-01 17:17:51,NaN,NaN
4,user_954,2019-01-01 21:31:18,NaN,NaN
...,...,...,...,...
991,user_965,2019-04-20 12:55:41,800.0,6927.0
992,user_967,2019-04-20 14:59:36,10154.0,NaN
993,user_973,2019-04-20 17:09:56,10254.0,NaN
994,user_977,2019-04-20 18:10:07,10354.0,NaN


### Преобразование в datetime

Ранее вы узнали, как преобразовывать столбец с датой в виде текста в дату в формате datetime.  Напомним, что это делается с помощью метода to_datetime() в библиотеке Pandas.

Внутрь метода нужно передать тот столбец, который нужно преобразовать.

Компьютеры понимают числа, но не понимают текст (хотя дата-сайентисты найдут, что на это ответить). Поэтому если у вас время записано в виде строки, нужно вытащить из него числа.

Благодаря этому вы можете получить много полезной информации в виде чисел: не только конкретное время (часы, минуты, секунды) и даты (год, месяц, день), но и более сложные вещи: например время дня, время года и др.

Если мы храним время в виде специальных объектов типа Timestamp, с ним можно проводить и другие операции, например, сложение и вычитание.

In [79]:
log['time'] = pd.to_datetime(log['time'])

In [ ]:
# пример
log = pd.read_csv("log.csv")
log = log.dropna()
log.columns = ["user_id", "time", "bet", "win"]
log["time"] = log["time"].apply(lambda x: x[1:])
log["time"] = pd.to_datetime(log["time"])
log['time'] = log.time.apply(lambda x: x.minute)
log["time"].head()

### Извлечение признаков времени

1. Повторяем Timestamp

Для начала вспомним, что мы можем делать с datetime.

Приведём примеры атрибутов, по которым мы можем обращаться к данным объектам:
* year возвращает год;
* month возвращает месяц;
* day возвращает день;
* hour, minute, second — час, минута, секунда;
* dayofweek — день недели от 0 до 6, где 0 — понедельник, 6 — воскресенье.

2. Повторяем apply/dt

Ранее в курсе вы разбирали метод apply(). Он позволяет применить какую-то функцию к каждому элементу в столбце.

В метод apply() можно передавать и обычные lambda-функции.

Например, мы хотим получить столбец, в котором каждым значением будет год из другого столбца. Это и есть Feature Engineering — создание новых признаков из старых. Мы можем сделать следующее:

year_column = log["time"].apply(lambda x: x.year)

Библиотека Pandas позволяет использовать аксессор dt для упрощения подобной работы:

year_column = log["time"].dt.year

Если вы попытаетесь обратиться к dt у столбца, в котором лежит что-то отличное от времени, вы получите ошибку.

Вы можете пользоваться любым из предложенных выше способов. 

3. Повторяем value_counts()

Одним из часто используемых методов в Pandas является value_counts().

Этот метод возвращает серию (Series), которая содержит количество уникальных элементов.

value_counts() возвращает значения отсортированными по убыванию.

Если в value_counts() передать значение параметра ascending=True, метод вернёт значения, отсортированные по возрастанию.

In [12]:
#year_column = log['time'].apply(lambda x: x.year)
year_column = log["time"].dt.year

In [16]:
test = pd.Series([1, 1, 1, 2, 3, 4, 4])
test.value_counts()

1    3
4    2
2    1
3    1
dtype: int64

In [19]:
minute_column = log["time"].dt.minute
test = pd.Series(minute_column)
#test.value_counts()

In [22]:
month_column = log["time"].dt.month
test = pd.Series(month_column)
test.value_counts(ascending=True)

4    170
2    259
3    264
1    292
Name: time, dtype: int64

In [28]:
dayofweek_column = log["time"].dt.dayofweek
test = pd.Series(dayofweek_column)
test.value_counts()

5    152
1    150
2    150
4    135
0    135
3    132
6    131
Name: time, dtype: int64

In [34]:
log = pd.read_csv('data/log.csv', header = None)
log.columns = ["user_id", "time", "bet", "win"]
log = log.dropna()
log["time"] = log["time"].apply(lambda x: x[1:])
log["time"] = pd.to_datetime(log["time"])
log['hour'] = log.time.apply(lambda x: x.hour)

                              user_id                time        bet  \
14   Запись пользователя № - user_917 2019-01-02 08:57:36   145732.0   
29   Запись пользователя № - user_942 2019-01-04 13:59:42  1678321.0   
151  Запись пользователя № - user_982 2019-01-16 21:54:22      100.0   
189  Запись пользователя № - user_964 2019-01-21 18:34:44      200.0   
205  Запись пользователя № - user_931 2019-01-22 05:26:59      300.0   
..                                ...                 ...        ...   
967  Запись пользователя № - user_975 2019-04-19 22:25:15     1000.0   
971  Запись пользователя № - user_912 2019-04-20 10:35:49    10554.0   
972  Запись пользователя № - user_926 2019-04-20 10:35:50    10354.0   
976  Запись пользователя № - user_970 2019-04-20 10:35:54    10354.0   
991  Запись пользователя № - user_965 2019-04-20 12:55:41      800.0   

           win  hour  
14   1987653.0     8  
29   9876543.0    13  
151     4749.0    21  
189     4667.0    18  
205     4319.0     5

#### Пропуски

ЗАПОЛНЕНИЕ КОНСТАНТОЙ


Это значит, что каждый пропуск в столбце мы заполним одним и тем же числом.

Чтобы заполнить пропуски в столбце каким-то значением, можно использовать метод fillna() у самого столбца. Аргументом этого метода будет число, которое появится на месте пропусков.

In [80]:
#log = pd.read_csv('data/log.csv', header = None)
#log.columns = ["user_id", "time", "bet", "win"]

log["bet"] = log["bet"].fillna(0)

In [55]:
test = pd.Series(log["bet"])
test.value_counts()

0.0          513
500.0         49
100.0         48
300.0         42
800.0         41
200.0         40
400.0         40
700.0         38
600.0         37
1000.0        35
900.0         28
9754.0        10
10554.0        8
10154.0        7
10254.0        7
9954.0         6
10654.0        5
10754.0        4
10354.0        4
10054.0        3
10454.0        3
9854.0         2
8734.0         1
12945.0        1
27000.0        1
12548.0        1
9876.0         1
156789.0       1
950.0          1
8700.0         1
7650.0         1
123981.0       1
5000.0         1
1678321.0      1
98753.0        1
145732.0       1
104540.0       1
Name: bet, dtype: int64

ЗАПОЛНЕНИЕ С ПОМОЩЬЮ ФУНКЦИИ

Теперь поработаем с признаком win, в котором тоже есть пропуски.

Иногда нужно заполнять пропуски не одним и тем же числом, а разными, в зависимости от какого-то условия. Перед нами именно этот случай.

Давайте напишем метод, который заполнит пропуски в признаке win в соответствии с предположением выше. 

Для этого можно применить метод apply() ко всей таблице и передать ему функцию, которая вычисляет размер выигрыша (или проигрыша) по следующей схеме:

Если значение в столбце win существует (не пропуск), вернуть это же значение. Это значит, что человек выиграл.
Если вместо значения в столбце win и в столбце bet — пропуски, вернуть 0.
Если в столбце bet нет пропуска, а в столбце win есть пропуск, вернуть отрицательное значение столбца bet (проигрыш).
На выходе получится таблица без пропусков в столбце win. Следующим шагом заменим старый столбец win на новый.

In [85]:
log = pd.read_csv('data/log.csv', header = None)
log.columns = ["user_id", "time", "bet", "win"]

log.bet=log['bet'].fillna(0)
log.win=log['win'].fillna(0)

def fillna_win(row):
    if row.win>0:
        return row.win
    elif row.bet==0:
        row.win=0
        return row.win
    elif row.bet>0 and row.win==0:
        return -row.bet  
  
# Применяем функцию   
log['win'] = log.apply(lambda row: fillna_win(row), axis=1)
log.head(20)

,user_id,time,bet,win
980,#error,NaN,10454.0,30117.0
981,#error,NaN,800.0,7035.0
982,#error,NaN,900.0,-900.0
983,#error,NaN,500.0,-500.0
984,Запись пользователя № - user_925,[2019-04-20 12:55:00,1000.0,-1000.0
985,#error,NaN,900.0,-900.0
986,#error,NaN,10454.0,29972.0
987,#error,NaN,10554.0,31634.0
988,#error,NaN,1000.0,-1000.0
989,Запись пользователя № - user_953,[2019-04-20 12:55:39,9954.0,-9954.0


In [58]:
log[log['win']<0].count()

user_id    347
time       339
bet        347
win        347
dtype: int64

Напомним, что добавить новый столбец (признак) в Pandas очень просто: нужно обратиться к нему по имени и присвоить значение.

log['net'] = # здесь должен быть код, создающий новый признак  

Кроме этого, в Pandas (как и в numpy) столбцы можно складывать друг с другом поэлементно.

In [86]:
log['net'] = log['win']-log['bet']
log[log['net']>0].median()

C:\Users\User\AppData\Local\Temp/ipykernel_10780/371600001.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  log[log['net']>0].median()


bet     600.0
win    5935.0
net    5347.0
dtype: float64

In [100]:
log.bet=log['bet'].fillna(0)
log.win=log['win'].fillna(0)

def fillna_net(row):
    if row.win<0:
        row.net=0.0
        return row.net
    elif row.win>0:
        return row.win-row.bet
  
# Применяем функцию   
log['net2'] = log.apply(lambda row: fillna_net(row), axis=1)
log.tail(20)

,user_id,time,bet,win,net,net2
980,#error,NaN,10454.0,30117.0,19663.0,19663.0
981,#error,NaN,800.0,7035.0,6235.0,6235.0
982,#error,NaN,900.0,-900.0,-1800.0,0.0
983,#error,NaN,500.0,-500.0,-1000.0,0.0
984,Запись пользователя № - user_925,[2019-04-20 12:55:00,1000.0,-1000.0,-2000.0,0.0
985,#error,NaN,900.0,-900.0,-1800.0,0.0
986,#error,NaN,10454.0,29972.0,19518.0,19518.0
987,#error,NaN,10554.0,31634.0,21080.0,21080.0
988,#error,NaN,1000.0,-1000.0,-2000.0,0.0
989,Запись пользователя № - user_953,[2019-04-20 12:55:39,9954.0,-9954.0,-19908.0,0.0


In [66]:
import numpy as np

# Как можно посчитать среднее значение для столбца bet, не учитывая при подсчёте пропуски?

log.bet.mean()
log.bet.sum() / log.bet.dropna().shape[0]
np.mean(log.bet)
log['bet'].dropna().mean()

3291.083

Оцениваем выигрыши и проигрыши

Посмотрим, сколько в среднем люди выигрывают и проигрывают.

In [87]:
# Посчитайте, какой процент посещений букмекерской конторы оборачивался ставкой. 
# Для этого поделите количество ставок (значений больше 0) на общее количество посещений конторы.

log[log['bet']>0].count() / log.count()

user_id    0.485000
time       0.479188
bet        0.485000
win        0.485000
net        0.485000
dtype: float64

In [101]:
# Каково среднее значение ставки (из столбца bet) в тех случаях, когда ставка была сделана?

log[log['bet']>0]['bet'].mean()

6785.738144329897

In [102]:
# Каков средний выигрыш (из столбца net) в тех случаях, когда ставка была сделана?
# Выигрышем будем считать любое изменение количества денег, в том числе отрицательное (в таком случае это проигрыш).

log[log['bet']>0].mean()

C:\Users\User\AppData\Local\Temp/ipykernel_10780/774994742.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  log[log['bet']>0].mean()


bet      6785.738144
win     24794.554639
net     18008.816495
net2    22834.969072
dtype: float64

In [103]:
# Каков средний размер потерь при проигрыше (из столбца net)?

log[log['win']<0].mean()

C:\Users\User\AppData\Local\Temp/ipykernel_10780/400618083.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  log[log['win']<0].mean()


bet     3372.743516
win    -3372.743516
net    -6745.487032
net2       0.000000
dtype: float64

In [114]:
#Посчитайте, чему равна минимальная ставка.
#Посчитайте, сколько раз была сделана минимальная ставка и запишите результат в переменную min_bet_amount в виде целого числа.


min_bet = log[log['bet']>0]['bet'].min()
min_bet_amount = log[log['bet']==min_bet]['bet'].count()

48

Объединяем датасеты

Merge

Теперь объединим данные с помощью метода pd.merge():

Первые два аргумента — таблицы, которые нужно будет объединить.

Третий аргумент — название признака, по которому будем объединять данные. Мы уже привели данные к одинаковому виду, и теперь их можно объединить по признаку 'user_id', чтобы получить полную информацию о пользователе. 

In [115]:
log = pd.read_csv('data/log.csv', header = None)
log.columns = ["user_id", "time", "bet", "win"]
users = pd.read_csv('data/users.csv', sep="\t", encoding="koi8_r")
users.columns = ["user_id", "email", "geo"]

# Приведём признак user_id к одному формату в обоих датасетах
users.user_id = users.user_id.apply(lambda x: x.lower())
# Избавимся от ошибок в user_id
log = log[log.user_id != "#error"]
log.user_id = log.user_id.str.split(" - ").apply(lambda x: x[1])

In [119]:
pd.merge(log, users, on='user_id')  

,user_id,time,bet,win,email,geo
0,user_919,[2019-01-01 14:06:51,NaN,NaN,Chikkaverle@icloud.com,Хабаровск
1,user_919,[2019-01-30 10:06:00,NaN,NaN,Chikkaverle@icloud.com,Хабаровск
2,user_919,[2019-02-05 14:33:44,NaN,NaN,Chikkaverle@icloud.com,Хабаровск
3,user_919,[2019-02-14 11:38:05,NaN,NaN,Chikkaverle@icloud.com,Хабаровск
4,user_919,[2019-03-02 4:23:36,300.0,NaN,Chikkaverle@icloud.com,Хабаровск
...,...,...,...,...,...,...
970,user_932,[2019-02-24 22:40:06,NaN,NaN,BraceWalker@bk.ru,Красноярск
971,user_932,[2019-03-15 10:56:14,NaN,NaN,BraceWalker@bk.ru,Красноярск
972,user_932,[2019-03-18 10:13:24,NaN,NaN,BraceWalker@bk.ru,Красноярск
973,user_932,[2019-03-27 12:18:24,NaN,NaN,BraceWalker@bk.ru,Красноярск


Groupby

Теперь повторим groupby.

Данный метод позволяет сгруппировать данные и применить к ним методы агрегации:

В данном случае мы группируем данные по признаку user_id.

После этого в каждой группе выбираем признак win.

Затем берём медиану по каждой группе по признаку win и на выходе получаем таблицу, где индексом является признак user_id. В этой таблице единственный столбец — медиана по каждой группе (то есть по каждому пользователю).

Наконец, последний вызов median() даёт нам медиану по предыдущему столбцу, то есть возвращает одно число.

In [121]:
log_user = pd.merge(log, users, on='user_id')  
log_user.groupby('user_id').win.median().median() 

5951.75

In [131]:
# Мой вариант

log = pd.read_csv('data/log.csv', header = None)
log.columns = ["user_id", "time", "bet", "win"]
users = pd.read_csv('data/users.csv', sep="\t", encoding="koi8_r")
users.columns = ["user_id", "email", "geo"]

def correct_user_id (new_id):
    if new_id == '#error':
        return ''
    else:
        return new_id[24:]

log = log.query('user_id != "#error"')
log['user_id'] = log.user_id.apply(correct_user_id)
log.time = log.time.str.strip('[')

log.drop_duplicates(subset=['time','user_id'])

log['time'] = pd.to_datetime(log['time'])

log.bet=log['bet'].fillna(0)
log.win=log['win'].fillna(0)

def fillna_win(row):
    if row.win>0:
        return row.win
    elif row.bet==0:
        row.win=0
        return row.win
    elif row.bet>0 and row.win==0:
        return -row.bet  
   
log['win'] = log.apply(lambda row: fillna_win(row), axis=1)

def fillna_net(row):
    if row.win<0:
        row.net=0.0
        return row.net
    elif row.win>0:
        return row.win-row.bet
 
log['net'] = log.apply(lambda row: fillna_net(row), axis=1)

In [135]:
import pandas as pd

log = pd.read_csv('data/log.csv', header = None)
log.columns = ["user_id", "time", "bet", "win"]
users = pd.read_csv('data/users.csv', sep="\t", encoding="koi8_r")
users.columns = ["user_id", "email", "geo"]

log = log[log.user_id != '#error']  
log.time = log.time.str.strip('[')
log.user_id = log.user_id.str.split(' - ').apply(lambda x: x[1])
users.user_id = users.user_id.apply(lambda x: x.lower()) 

log.drop_duplicates(subset=['time','user_id'])

log['time'] = pd.to_datetime(log['time'])

log.bet=log['bet'].fillna(0)
log.win=log['win'].fillna(0)

log['net'] = log['win'] - log['bet']

In [ ]:
def fillna_win(row):
    if row.win>0:
        return row.win
    elif row.bet==0:
        row.win=0
        return row.win
    elif row.bet>0 and row.win==0:
        return -row.bet  
   
log['win'] = log.apply(lambda row: fillna_win(row), axis=1)

In [136]:
# Какова медиана баланса по каждому пользователю?
# Для решения задачи сделайте группировку по пользователям, возьмите признак net, просуммируйте по каждому пользователю и возьмите медиану.

log.groupby('user_id').net.sum().median()

2202.0

In [ ]:
# Во сколько раз различаются максимальное и минимальное значения средней ставки по городам?

df = pd.merge(log, users, on='user_id')

group = df.groupby('geo').bet.mean().sort_values()
(group.max())/(group.min())

In [130]:
log.groupby('user_id').net.sum().median() 

5709.0